# 2017 Game Sales Outlook <a id='back'></a> 

* [Introduction](#intro)
* [1 Data Overview and Preprocessing](#data_over)
    * [1.1 Imports and Load Data](#load)
    * [1.2 Data Preprocessing](#data_pre)
    * [1.8 Aggregate Data](#agg_dat)
    * [1.9 Preliminary Conclusion](#conc_prelim)
* [2 Data Analysis](#analysis)
    * [2.1 User Count](#user_stud)
    * [2.6  Study Conclusion](#conc_stud)
* [3 Test Statistical Hypotheses](#test_hyp)
    * [3.1 First Hypothesis](#hyp_1)
    * [3.2 Second Hypothesis](#hyp_2)
* [4 Final Conclusion](#fin_conc)

# Introduction <a id='intro'></a>
    
lorem ipsum croque monsieur asta la vista baby

test the following hypotheses:
* Average user ratings of the Xbox One and PC platforms are the same.
* Average user ratings for the Action and Sports genres are different.


## Data Overview and Preprocessing <a id='data_over'></a>

In [20]:
# import pandas, a general data-management library
import pandas as pd

# import numpy, a general statistics library
import numpy as np

# import pyplot, a graph plotting library
from matplotlib import pyplot as plt 

# import scipy, a statistical analysis library
from scipy import stats as st

# import plotly.express, a high level plotting library
import plotly.express as px

In [2]:
# load the data into a pandas dataframe
df = pd.read_csv('games.csv')

In [3]:
#print general information about the dataset
df.info()
display(df.describe())
display(df.sample(n=10, random_state=0))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Year_of_Release,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
7634,Press Your Luck 2010 Edition,DS,2009.0,Misc,0.18,0.00,0.00,0.01,NaN,tbd,E
13771,Aeon Flux,PS2,2005.0,Action,0.02,0.02,0.00,0.01,66.0,5.8,T
3051,Castlevania: Lords of Shadow,X360,2010.0,Action,0.42,0.17,0.01,0.05,83.0,7.8,M
15726,Prince of Stride,PSV,2015.0,Adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN
578,Final Fantasy XIII-2,PS3,2011.0,Role-Playing,0.78,0.73,0.89,0.23,79.0,6.6,T
14668,World of Zoo,PC,2009.0,Simulation,0.00,0.02,0.00,0.01,NaN,8.4,E
10421,Gravity Games Bike: Street Vert Dirt,PS2,2002.0,Sports,0.05,0.04,0.00,0.01,24.0,4.1,T
10231,Calling,Wii,2009.0,Adventure,0.06,0.04,0.00,0.01,49.0,6.7,T
12163,Titanic Mystery,DS,2010.0,Puzzle,0.05,0.01,0.00,0.01,NaN,tbd,T
1090,PGR: Project Gotham Racing 2,XB,2003.0,Racing,0.97,0.59,0.04,0.07,NaN,NaN,NaN


## Data Overview Summary 

We can see we have a few issues with our data which will require preprocessing before we can perform our analysis.

These issues amount to:
* column names in improper case
* missing values, and values in a numeric-type data field which are of an indeterminate value. To be specific, these are the 'tbd' values in our Critic_Score and User_Score columns.

Further, we will create a few new columns using data from those existing to aid our analysis.

[Back to Contents](#back)

# Data Preprocessing <a id='data_pre'></a>

First, we will handle the simple task of reassigning our column names to proper snake_case.

In [31]:
# rename columns to proper snake_case
df.columns= df.columns.str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   Int64  
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.6+ MB


Next, we will process the missing values.

We will address this by-column as-follows:
* name  - fill with 'noname' as the name of the game is not pertinent to our analysis
* year_of_release - drop all entries with null values as the game's release year is necessary for our forecast and filling these values could alter the results of the analysis.
* critic_score  - Leave these as NA since filling them will alter minor elements of our analysis
* user_score    - Leave these as NA since filling them will alter minor elements of our analysis
* rating        - Leave these as NA since filling them will alter minor elements of our analysis

In [28]:
# For curiosity's sake we will take a look at the two n/a names column values
display(df[(df['name'].isnull())])

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993,NaN,1.78,0.53,0.00,0.08,<NA>,NaN,NaN
14244,NaN,GEN,1993,NaN,0.00,0.00,0.03,0.00,<NA>,NaN,NaN


Since these entries are for games that are fifteen years old, were released for a console which is no longer supported(Sega Genesis), and are missing their genre data(one of our variables of interest), we can drop these entries without altering the results of our analysis or forecast.

In [33]:
# drop entries with null values in the name column
df.dropna(subset='name', inplace=True)

#drop entries with null values in the year_of_release column
df.dropna(subset='year_of_release', inplace=True)

#verify total entries to non-null entries in dropped columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   Int64  
 9   user_score       7463 non-null   float64
 10  rating           9768 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.5+ MB


We can see our total number of entries, 16444, matches our number of entries for our primary columns-of-interest.

Now we will proceed with setting our columns to the proper datatype.

In [34]:
# convert data to proper filetypes
# using this method of conversion will throw an error if there are decimal values
df['year_of_release'] = df['year_of_release'].astype('Int64')
df['critic_score'] = df['critic_score'].astype('Int64')

# convert the user score column to float since it has decimal values
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

df.info()
display(df.sample(n=10, random_state=0))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   Int64  
 9   user_score       7463 non-null   float64
 10  rating           9768 non-null   object 
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.5+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
12857,MotoGP 4 - Official Game of MotoGP,PS2,2005,Racing,0.03,0.02,0.00,0.01,<NA>,NaN,NaN
14457,Stacked with Daniel Negreanu,XB,2006,Misc,0.02,0.01,0.00,0.00,61,NaN,T
6491,Riding Spirits,PS2,2002,Racing,0.13,0.10,0.00,0.03,59,9.0,E
6134,Kung Fu Chaos,XB,2003,Fighting,0.21,0.06,0.00,0.01,68,8.5,T
1185,Mario Party 7,GC,2005,Misc,0.95,0.11,0.46,0.04,64,7.9,E
11096,Pachitte Chonmage Tatsujin 10: Pachinko Fuyu n...,PS2,2007,Misc,0.00,0.00,0.09,0.00,<NA>,NaN,NaN
12447,2 Games in 1: Sonic Pinball Party & Columns Crown,GBA,2005,Misc,0.04,0.02,0.00,0.00,<NA>,NaN,NaN
9976,Sengoku Cyber: Fujimaru Jigokuhen,PS,1995,Strategy,0.00,0.00,0.11,0.01,<NA>,NaN,NaN
5463,NASCAR Thunder 2002,XB,2001,Racing,0.25,0.07,0.00,0.01,82,NaN,E
13905,Dora's Big Birthday Adventure,PS2,2010,Misc,0.02,0.01,0.00,0.00,<NA>,NaN,E


Now, we will create a new column 'global_sales' to aid our analysis later.

In [41]:
# create 'global_sales' column as a sum of the other *_sales columns
df['global_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

In [45]:
df_2010s = df[(df['year_of_release'] >= 2010)]
df_2010s.info()
display(df_2010s.sample(n=10, random_state=0))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5277 entries, 14 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             5277 non-null   object 
 1   platform         5277 non-null   object 
 2   year_of_release  5277 non-null   Int64  
 3   genre            5277 non-null   object 
 4   na_sales         5277 non-null   float64
 5   eu_sales         5277 non-null   float64
 6   jp_sales         5277 non-null   float64
 7   other_sales      5277 non-null   float64
 8   critic_score     2312 non-null   Int64  
 9   user_score       2499 non-null   float64
 10  rating           3113 non-null   object 
 11  global_sales     5277 non-null   float64
dtypes: Int64(2), float64(6), object(4)
memory usage: 546.3+ KB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,global_sales
15295,Gochuumon wa Usagi Desu ka?? Wonderful Party!,PSV,2016,Adventure,0.00,0.00,0.02,0.00,<NA>,NaN,NaN,0.02
81,FIFA Soccer 13,PS3,2012,Action,1.06,5.01,0.13,1.97,88,6.6,E,8.17
14823,Tiger & Bunny: On-Air Jack!,PSP,2012,Action,0.00,0.00,0.03,0.00,<NA>,NaN,NaN,0.03
12195,Sherlock Holmes: Crimes & Punishments,PS3,2014,Adventure,0.03,0.03,0.00,0.01,<NA>,NaN,NaN,0.07
6889,Dreamcast Collection,X360,2011,Misc,0.16,0.06,0.00,0.02,53,5.2,T,0.24
1806,Watch Dogs 2,PS4,2016,Action,0.37,0.58,0.00,0.18,83,7.8,M,1.13
4668,EA Sports UFC 2,XOne,2016,Sports,0.24,0.14,0.00,0.04,76,6.0,T,0.42
3685,F1 2013,PS3,2013,Racing,0.01,0.42,0.03,0.09,77,6.3,E,0.55
1314,Madden NFL 15,PS3,2014,Sports,1.08,0.11,0.00,0.27,<NA>,2.7,E,1.46
1037,NBA 2K15,XOne,2014,Sports,1.37,0.18,0.00,0.18,82,6.3,E,1.73


Now our preprocessing is complete.

We have prepared the dataset for analysis, accounted for the missing data which could cause substantial issues, and have created new columns using existing data to aid our analysis later. 

# Data Analysis  <a id='analysis'></a>

In [ ]:
plt = px.histogram(df, 'year_of_release', 'global_sales', color='platform', )
plt.show()

In [ ]:
plt = px.histogram(df_2010s,5454544 'year_of_release', 'global_sales', color='platform', )
plt.show()

In [7]:
# code

### Analysis Conclusion

[Back to Contents](#back)

# Test Hypotheses <a id='test_hyp'></a>

In [8]:
# test hyp1

quick hyp1 summary

In [9]:
# test hyp2

quick hyp2 summary

overall hyp test conclusion

# final conclusion <a id='fin_conc'></a>



[Back to Contents](#back)